In [35]:
import json
!pip install folium
import folium
import numpy as np
import pandas as pd
import datetime
from datetime import time, datetime, timedelta
from math import sin, cos, sqrt, atan2,dist

You should consider upgrading via the '/Users/kaiyuanhou/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
path_response = '[[40.791250000000005,-73.96791],[40.79081,-73.96686000000001],[40.79046,-73.96603],[40.790380000000006,-73.96579000000001],[40.79036000000001,-73.96578000000001],[40.79027000000001,-73.96556000000001],[40.78976,-73.96593],[40.789640000000006,-73.96599],[40.78951,-73.96611],[40.789030000000004,-73.96647],[40.788700000000006,-73.9667],[40.78824,-73.96702],[40.788120000000006,-73.96711],[40.788050000000005,-73.96693],[40.78799,-73.96679],[40.78792000000001,-73.96668000000001],[40.7877,-73.96640000000001],[40.787650000000006,-73.96636000000001],[40.78741,-73.96641000000001],[40.78692,-73.96651],[40.786530000000006,-73.96664000000001],[40.78633000000001,-73.96666],[40.786120000000004,-73.96666],[40.786,-73.96669],[40.785470000000004,-73.96683],[40.785410000000006,-73.96686000000001],[40.78538,-73.96694000000001],[40.785120000000006,-73.96687],[40.784940000000006,-73.96683],[40.78481,-73.96667000000001],[40.784560000000006,-73.96636000000001],[40.78443,-73.9662],[40.78417,-73.96572],[40.78403,-73.96543000000001],[40.783950000000004,-73.96551000000001],[40.7839,-73.96554],[40.78389000000001,-73.96552000000001],[40.783680000000004,-73.96479000000001],[40.78363,-73.96463],[40.7836,-73.96453000000001],[40.78352,-73.96458000000001],[40.783480000000004,-73.96448000000001],[40.78334,-73.96417000000001],[40.783280000000005,-73.96406],[40.78318,-73.96395000000001],[40.78305,-73.96383],[40.78284,-73.96371],[40.78244,-73.96362],[40.782210000000006,-73.96354000000001],[40.782050000000005,-73.96344],[40.781870000000005,-73.96332000000001],[40.78177,-73.96319000000001],[40.78163,-73.96301000000001],[40.7813,-73.9625],[40.78123,-73.96242000000001],[40.7811,-73.96235],[40.78085,-73.96214],[40.78049,-73.96178],[40.78025,-73.96149000000001],[40.78007,-73.96163],[40.77984,-73.96180000000001],[40.77958,-73.96199],[40.779500000000006,-73.96182],[40.77951,-73.96178],[40.77953,-73.96177],[40.7794,-73.96148000000001],[40.77929,-73.96121000000001]]'
path_response = json.loads(path_response)

In [3]:
def interpolate(start, end, interval=0.0001, result=None):
    if result is None:
        result = [start]
    if dist(start,end) < interval:
        result.append(end)
    else:
        direction = atan2(end[1]-start[1], end[0]-start[0])
        new_lat = start[0] + interval * cos(direction)
        new_lon = start[1] + interval * sin(direction)
        new_loc = [new_lat,new_lon]
        result.append(new_loc)
        interpolate(new_loc, end, interval=interval, result=result)
    return result

In [4]:
interpolate_path = []
for i in range(len(path_response)-1):
    interpolate_path = interpolate_path + interpolate(path_response[i], path_response[i+1], interval= 0.0001)

# remove duplicates    
interpolate_path = pd.DataFrame(interpolate_path).drop_duplicates().to_numpy()


In [5]:
# Plot list of latitudes and longitudes into a map
m = folium.Map()

# Add the latitudes and longitudes to the map as markers
for lat, lng in path_response:
    folium.Marker([lat, lng]).add_to(m)

# Fit the boundaries
path = np.array(path_response)
m.fit_bounds([np.array(path_response).min(axis=0).tolist(), np.array(path_response).max(axis=0).tolist()])

# Display the map
m

In [6]:
# Plot list of latitudes and longitudes into a map
m = folium.Map()

# Add the latitudes and longitudes to the map as markers
for lat, lng in interpolate_path:
    folium.Marker([lat, lng]).add_to(m)

# Fit the boundaries
path = np.array(interpolate_path)
m.fit_bounds([np.array(interpolate_path).min(axis=0).tolist(), np.array(interpolate_path).max(axis=0).tolist()])

# Display the map
m

In [29]:
df = pd.read_csv('./Datasets/dataset_encode.csv')

In [30]:
df[['date','time']] = df['Timestamp'].str.split(' ',expand=True)
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')
df['time'] = df.time.dt.time
df.head()

,Timestamp,Latitude,Longitude,Victim_AGE,Victim_RACE,Victim_Sex,Category,date,time
0,12/25 23:00:00,40.772875,-73.989542,0,0,0,0,12/25,23:00:00
1,12/15 04:15:00,40.807781,-73.945485,1,1,1,1,12/15,04:15:00
2,12/05 22:00:00,40.828851,-73.943834,1,1,1,0,12/05,22:00:00
3,12/10 15:30:00,40.816509,-73.946547,2,0,1,0,12/10,15:30:00
4,12/08 23:20:00,40.789633,-73.940068,1,1,0,1,12/08,23:20:00


In [43]:
current_time = datetime.now()
delta = timedelta(hours = 2)
start_time= (current_time - delta).time()
end_time = (current_time + delta).time()
rad = 0.001 # 100m
# lat_center = 40.781
# lng_center = -73.953

lat_center = 40.813
lng_center = -73.943

df[(df['time'] >= start_time) & (df['time'] <= end_time) & (df['Latitude'] > lat_center-rad) & (df['Latitude'] < lat_center+rad) & (df['Longitude'] > lng_center-rad) & (df['Longitude'] < lng_center+rad)]

,Timestamp,Latitude,Longitude,Victim_AGE,Victim_RACE,Victim_Sex,Category,date,time
5445,12/08 14:15:00,40.813414,-73.943211,3,0,1,0,12/08,14:15:00
5508,11/28 14:00:00,40.812547,-73.942099,1,0,0,2,11/28,14:00:00
6237,11/22 13:40:00,40.813414,-73.943211,4,1,0,2,11/22,13:40:00
7610,11/15 13:00:00,40.812209,-73.942251,3,0,1,2,11/15,13:00:00
11806,10/23 16:00:00,40.812547,-73.942099,0,0,0,0,10/23,16:00:00
...,...,...,...,...,...,...,...,...,...
1069514,12/11 13:00:00,40.812797,-73.943663,3,0,1,0,12/11,13:00:00
1069656,08/20 16:00:00,40.812547,-73.942099,1,0,0,0,08/20,16:00:00
1071354,06/24 16:00:00,40.813414,-73.943211,1,0,1,0,06/24,16:00:00
1073194,08/20 15:00:00,40.812547,-73.942099,1,0,0,0,08/20,15:00:00


In [116]:
def compute_value(geolocation, df, rad=0.002, time_interval = 2):
    current_time = datetime.now()
    delta = timedelta(hours = time_interval)
    start_time= (current_time - delta).time()
    end_time = (current_time + delta).time()
    
    lat_center, lng_center = geolocation
    
    ret = df[\
        (df['time'] >= start_time) & (df['time'] <= end_time) \
       & (df['Latitude'] > lat_center-rad) & (df['Latitude'] < lat_center+rad) \
       & (df['Longitude'] > lng_center-rad) & (df['Longitude'] < lng_center+rad) \
            ]
    
    return ret


In [117]:
import time
start = time.time()
ret_arr = []
for loc in interpolate_path:
    ret_arr.append(len(compute_value(loc, df)))
print(f'time: {time.time() - start}')

time: 19.168816089630127


In [108]:
heatmap = []
for loc, count in zip(interpolate_path, ret_arr):
    heatmap.append([*loc, count])

In [114]:
pd.DataFrame(heatmap).to_csv('sample_heatmap.csv', index = None, header=None)

In [104]:
import pickle
with open('heatmap.pickle', 'wb') as f:
    pickle.dump(heatmap, f, protocol=pickle.HIGHEST_PROTOCOL)

In [105]:
with open('heatmap.pickle', 'rb') as f:
    b = pickle.load(f)
print(b)

[[array([ 40.79125, -73.96791]), 734], [array([ 40.79121135, -73.96781777]), 607], [array([ 40.7911727 , -73.96772554]), 607], [array([ 40.79113405, -73.96763331]), 613], [array([ 40.79109541, -73.96754108]), 657], [array([ 40.79105676, -73.96744885]), 722], [array([ 40.79101811, -73.96735662]), 665], [array([ 40.79097946, -73.96726439]), 665], [array([ 40.79094081, -73.96717216]), 627], [array([ 40.79090216, -73.96707993]), 612], [array([ 40.79086351, -73.9669877 ]), 611], [array([ 40.79082487, -73.96689548]), 592], [array([ 40.79081, -73.96686]), 592], [array([ 40.79077114, -73.96676786]), 598], [array([ 40.79073229, -73.96667571]), 617], [array([ 40.79069343, -73.96658357]), 617], [array([ 40.79065458, -73.96649143]), 617], [array([ 40.79061572, -73.96639929]), 546], [array([ 40.79057687, -73.96630714]), 546], [array([ 40.79053801, -73.966215  ]), 508], [array([ 40.79049916, -73.96612286]), 505], [array([ 40.7904603 , -73.96603072]), 511], [array([ 40.79046, -73.96603]), 511], [arra